In [1]:
from surface_dynamics.all import Origami
import numpy as np

In [2]:
def cycle_notation(array): 
    '''
        Argument: array of integers representing gluing instruction of origami, every element in array is glued to the next
        Return: string representing the cycle of the gluing
        Comment: assumes array is a permutation we are using which is a cycle as we are gluing the boxes
            until the edges meet, and is therefore NOT a general method for converting permutation to cycle notation, 
            it basically makes the array a string
    '''
    cycle = '('
    for i in array:
        cycle = cycle + str(int(i)) + ','
    cycle = cycle[:-1] + ')'
    return cycle

In [3]:
def enum_strata3(n):
    '''
        Argument: integer n, size of origamis
        Return: array of origamis of the form type 3 in H(4) that have size n
        Comment: https://jamboard.google.com/d/1OSRaahlM2KiY1_onnQ5y6TDBOrseFGrHhtssKQc2ee0/edit?usp=sharing
            on slide 4 explains external gluing formula
    '''
    surfaces = []
    for u2 in range(4, n): # base rectangle has width [4, n-1)
        for h2 in range(1, n // u2 + 1): # base rectangle has height [1, n/u2)
            # top rectangle has width at most u2 - 3 or at most area of the top rectangle
            upperbound = min(u2 - 2, n - u2 * h2 + 1) 
            for u1 in range(1, upperbound): # top rectangle has width [1, upperbound)
                if (n - u2 * h2) % u1 == 0: # as long as u1 divides the area, it's valid
                    h1 = (n - u2 * h2) // u1
                    rectangle2 = np.arange(1, u2 * h2 + 1)
                    rect2 = np.reshape(rectangle2, (h2, u2)) # base rectangle rect2
                    rect2 = np.flip(rect2,0)
                    rectangle1 = np.arange(u2*h2 + 1, n + 1)
                    rect1 = np.reshape(rectangle1, (h1, u1)) # top rectangle rect1
                    rect1 = np.flip(rect1,0)
                    hperm = '' 
                    # horizontal (right) permutation is independent of twist
                    for i in range(h1):
                        hperm += cycle_notation(rect1[i])
                    for i in range(h2):
                        hperm += cycle_notation(rect2[i])
                    # width of smaller rectangles
                    for w1 in range(1, u2 - u1 - 1): # w1 is between 1 and u2-u1-2
                        for w2 in range(1, u2 - u1 - w1): # w2 is between 1 and u2-u1-w1-1
                            w3 = u2 - u1 - w1 - w2
                            for t1 in range(u1): # singularity on cylinder 1
                                for t2 in range(u2): # singularity on cylinder 2
                                    marker = [1] * u2
                                    vperm = ''
                                    while sum(marker) != 0:
                                        nextn = np.nonzero(marker)[0][0]
                                        column = []
                                        while marker[nextn] != 0:
                                            # print("nextn: ", nextn)
                                            column = np.concatenate(([row[nextn] for row in rect2],column))
                                            marker[nextn] = 0
                                            if nextn < u1:
                                                col = [row[nextn] for row in rect1]
                                                column = np.concatenate((col, column))
                                                nextn = (t2+(u1+nextn-t1)%u1)%u2
                                            else:
                                                offset = 0
                                                # print("ofs: ", offset)
                                                if nextn - u1 >= w2 + w1:
                                                    offset += (nextn - u1 - w2 - w1)
                                                    # print("ofs: ", offset)
                                                else:
                                                    offset += w3
                                                    # print("ofs: ", offset)
                                                    if nextn - u1 >= w1:
                                                        offset += (nextn - u1 - w1)
                                                        # print("ofs: ", offset)
                                                    else:
                                                        offset += (w2 + nextn - u1)
                                                        # print("ofs: ", offset)
                                                nextn = (t2+u1+offset)%u2
                                        vperm += cycle_notation(np.flip(column,0))
                                    origami = Origami(hperm, vperm)
                                    # print(origami)
                                    # print(origami.stratum())
                                    # origami.show()
                                    surfaces.append(origami)
    return surfaces

In [4]:
def enum_strata4(n):
    '''
        Argument: integer n, size of origamis
        Return: array of origamis of the form type 4 in H(4) that have size n
        Comment: https://jamboard.google.com/d/1OSRaahlM2KiY1_onnQ5y6TDBOrseFGrHhtssKQc2ee0/edit?usp=sharing
            on slide 5 explains external gluing formula
    '''
    surfaces = []
    for u2 in range(3, n - 1): # base rectangle has width [3, n-2), because top rectangle is at least 2
        for h2 in range(1, n // u2 + 1): # base rectangle has height [1, n/u2)
            # top rectangle has width at most u2 - 3 or at most area of the top rectangle
            for v in range(1, u2 - 1): # v is in [1, u2-2]
                for u1 in range(v + 1, n - u2 * h2 + 1): # top rectangle has width [v+1, area)
                    if (n - u2 * h2) % u1 == 0: # as long as u1 divides the area, it's valid
                        h1 = (n - u2 * h2) // u1
                        rectangle2 = np.arange(1, u2 * h2 + 1)
                        rect2 = np.reshape(rectangle2, (h2, u2)) # base rectangle rect2
                        rect2 = np.flip(rect2,0)
                        rectangle1 = np.arange(u2*h2 + 1, n + 1)
                        rect1 = np.reshape(rectangle1, (h1, u1)) # top rectangle rect1
                        rect1 = np.flip(rect1,0)
                        hperm = '' 
                        # horizontal (right) permutation is independent of twist
                        for i in range(h1):
                            hperm += cycle_notation(rect1[i])
                        for i in range(h2):
                            hperm += cycle_notation(rect2[i])
                        # width of smaller rectangles
                        for w1 in range(1, u2 - v): # w1 is between 1 and u2-v-1
                            w2 = u2 - v - w1
                            for t1 in range(u1): # singularity on cylinder 1
                                for t2 in range(u2): # singularity on cylinder 2
                                    marker1 = [1] * u1 # marks columns in rect1 that have been visited by 0
                                    marker2 = [1] * u2
                                    vperm = ''
                                    while sum(marker2) != 0:
                                        nextn = np.nonzero(marker2)[0][0]
                                        column = []
                                        while marker2[nextn] != 0:
                                            # print("nextn: ", nextn)
                                            column = np.concatenate(([row[nextn] for row in rect2],column))
                                            marker2[nextn] = 0
                                            if nextn >= v:
                                                if (nextn - v)%u2 >= w1:
                                                    nextn = (t2 + nextn - w1)%u2
                                                else:
                                                    nextn = (t2 + w2 + nextn)%u2
                                            else:
                                                offset = (u1 - v + nextn)%u1
                                                while (offset + u1 - t1)%u1 >= v and marker1[offset] != 0:
                                                    col = [row[offset] for row in rect1]
                                                    column = np.concatenate((col, column))
                                                    marker1[offset] = 0
                                                    offset = (offset + u1 - t1 - v)%u1
                                                if marker1[offset] == 0:
                                                    break
                                                col = [row[offset] for row in rect1]
                                                column = np.concatenate((col, column))
                                                marker1[offset] = 0
                                                nextn = (t2+(offset+u1-t1)%u1)%u2
                                        vperm += cycle_notation(np.flip(column,0))
                                    while sum(marker1) != 0:
                                        offset = np.nonzero(marker1)[0][0]
                                        column = []
                                        while marker1[offset] != 0:
                                            # print("nextn: ", nextn)
                                            column = np.concatenate(([row[offset] for row in rect1],column))
                                            marker1[offset] = 0
                                            if (offset + u1 - t1)%u1 >= v:
                                                offset = (offset + u1 - t1 - v)%u1
                                            else:
                                                nextn = (t2+(offset+u1-t1)%u1)%u2
                                                while nextn >= v and marker2[nextn] != 0:
                                                    column = np.concatenate(([row[nextn] for row in rect2],column))
                                                    marker2[nextn] = 0
                                                    if (nextn - v)%u2 >= w1:
                                                        nextn = (t2 + nextn - w1)%u2
                                                    else:
                                                        nextn = (t2 + w2 + nextn)%u2
                                                if marker2[nextn] == 0:
                                                        break
                                                offset = (u1 - v + nextn)%u1
                                        vperm += cycle_notation(np.flip(column,0))
                                    origami = Origami(hperm, vperm)
                                    # print(origami)
                                    # print(origami.stratum())
                                    # origami.show()
                                    surfaces.append(origami)
    return surfaces

In [5]:
#for ori in enum_strata3(7):
    #print(ori.stratum())
    #print(ori)
    #ori.show()

In [6]:
marker1 = []
marker2 = []
marker3 = []
rect1 = []
rect2 = []
rect3 = []
t1 = 0
t2 = 0
t3 = 0
u1 = 0
u2 = 0
u3 = 0

def s1cylinder1(offset1, currentperm):
    '''
    Argument: the offset to start with, 
              the current accumulating permutation
    Return: the permutation in list(array) form BEFORE flip
    Comment: the parameters are supposed to be passed by REFERENCE
    '''
    global marker1
    global rect1
    global t1
    global u1
    if marker1[offset1] == 0:
        return currentperm
    permutation = np.concatenate(([row[offset1] for row in rect1],currentperm))
    marker1[offset1] = 0
    offset2 = (offset1 - t1 + u1)%u1
    return s1cylinder2(offset2, permutation)
    
def s1cylinder2(offset2, currentperm):
    global marker2
    global rect2
    global t2
    global t3
    global u1
    global u2
    global u3
    if marker2[offset2] == 0:
        return currentperm
    permutation = np.concatenate(([row[offset2] for row in rect2],currentperm))
    marker2[offset2] = 0
    if (offset2 - t2 + u2)%u2 < u1:
        offset1 = (offset2 - t2 + u2)%u2
        return s1cylinder1(offset1, permutation)
    else:
        offset3 = ((offset2 - t2 + u2)%u2 - u1 + t3 + u3 - (u2 - u1))%u3
        return s1cylinder3(offset3, permutation)

def s1cylinder3(offset3, currentperm):
    global marker3
    global rect3
    global t3
    global u1
    global u2
    global u3
    if marker3[offset3] == 0:
        return currentperm
    permutation = np.concatenate(([row[offset3] for row in rect3],currentperm))
    marker3[offset3] = 0
    if offset3 < u2 - u1:
        offset2 = u1 + offset3
        return s1cylinder2(offset2, permutation)
    else:
        offset3 = (t3 + offset3 - (u2 - u1))%u3
        return s1cylinder3(offset3, permutation)

def enum_strata1(n):
    '''
        Argument: integer n, size of origamis
        Return: array of origamis of the form type 1 in H(4) that have size n
        Comment: https://jamboard.google.com/d/1OSRaahlM2KiY1_onnQ5y6TDBOrseFGrHhtssKQc2ee0/edit?usp=sharing
            on slide 6 explains external gluing formula
    '''
    surfaces = []
    global u2
    for u2 in range(2, n): # middle rectangle has width [2, n-1)
        for h2 in range(1, n // u2 + 1): # middle rectangle has height [1, n/u2)
            # top rectangle has width at most u2 - 1 or at most area left - 2
            global u1
            for u1 in range(1, min(u2, n - u2 * h2 - 1)):
                for h1 in range(1, (n - u2 * h2) // u1 + 1): # top rectangle has width [1, area)
                    global u3
                    for u3 in range(u2 - u1 + 1, n - u2 * h2 - u1 * h1 + 1):
                        if (n - u2 * h2 - u1 * h1) % u3 == 0: # as long as u1 divides the area, it's valid
                            h3 = (n - u2 * h2 - u1 * h1) // u3
                            rectangle2 = np.arange(1, u2 * h2 + 1)
                            global rect2
                            rect2 = np.reshape(rectangle2, (h2, u2)) # base rectangle rect2
                            rect2 = np.flip(rect2,0)
                            rectangle1 = np.arange(u2 * h2 + 1, u2 * h2 + u1 * h1 + 1)
                            global rect1
                            rect1 = np.reshape(rectangle1, (h1, u1)) # top rectangle rect1
                            rect1 = np.flip(rect1,0)
                            rectangle3 = np.arange(u2 * h2 + u1 * h1 + 1, n + 1)
                            global rect3
                            rect3 = np.reshape(rectangle3, (h3, u3)) # top rectangle rect1
                            rect3 = np.flip(rect3,0)
                            hperm = '' 
                            # horizontal (right) permutation is independent of twist
                            for i in range(h1):
                                hperm += cycle_notation(rect1[i])
                            for i in range(h2):
                                hperm += cycle_notation(rect2[i])
                            for i in range(h3):
                                hperm += cycle_notation(rect3[i])
                            global t1
                            for t1 in range(u1): # singularity on cylinder 1
                                global t2
                                for t2 in range(u2 - u1): # singularity on cylinder 2
                                    global t3
                                    for t3 in range(u3):
                                        global marker1
                                        marker1 = [1] * u1 # marks columns in rect1 that have been visited by 0
                                        global marker2
                                        marker2 = [1] * u2
                                        global marker3
                                        marker3 = [1] * u3
                                        vperm = ''
                                        while sum(marker2) != 0:
                                            offset2 = np.nonzero(marker2)[0][0]
                                            column = []
                                            permutation = s1cylinder2(offset2, column)
                                            vperm += cycle_notation(np.flip(permutation,0))
                                        while sum(marker1) != 0:
                                            offset1 = np.nonzero(marker1)[0][0]
                                            column = []
                                            permutation = s1cylinder1(offset1, column)
                                            vperm += cycle_notation(np.flip(permutation,0))
                                        while sum(marker3) != 0:
                                            offset3 = np.nonzero(marker3)[0][0]
                                            column = []
                                            permutation = s1cylinder3(offset3, column)
                                            vperm += cycle_notation(np.flip(permutation,0))
                                        origami = Origami(hperm, vperm)
#                                         print(origami)
#                                         print(t1, t2, t3)
#                                         print(origami.stratum())
#                                         origami.show()
                                        surfaces.append(origami)
    return surfaces

In [7]:
# for i in range(15):
#     for ori in enum_strata1(i):
#         print(ori.stratum())

In [8]:
marker1 = []
marker2 = []
marker3 = []
rect1 = []
rect2 = []
rect3 = []
t1 = 0
t2 = 0
t3 = 0
u1 = 0
u2 = 0
u3 = 0

def s2cylinder1(offset1, currentperm):
    '''
    Argument: the offset to start with, 
              the current accumulating permutation
    Return: the permutation in list(array) form BEFORE flip
    Comment: the parameters are supposed to be passed by REFERENCE
    '''
    global marker1
    global rect1
    global t1
    global u1
    if marker1[offset1] == 0:
        return currentperm
    permutation = np.concatenate(([row[offset1] for row in rect1],currentperm))
    marker1[offset1] = 0
    offset2 = ((offset1 - t1 + u1)%u1 + t2 + u2 - u1)%u2
    return s2cylinder2(offset2, permutation)
    
def s2cylinder2(offset2, currentperm):
    global marker2
    global rect2
    global t2
    global t3
    global u1
    global u2
    global u3
    if marker2[offset2] == 0:
        return currentperm
    permutation = np.concatenate(([row[offset2] for row in rect2], currentperm))
    marker2[offset2] = 0
    if offset2 < u1:
        offset1 = offset2
        return s2cylinder1(offset1, permutation)
    elif offset2 < u2 - u3:
        offset2 = (offset2 - u1 + t2 + u3)%u2
        return s2cylinder2(offset2, permutation)
    else:
        offset3 = offset2 - (u2 - u3)
        return s2cylinder3(offset3, permutation)

def s2cylinder3(offset3, currentperm):
    global marker3
    global rect3
    global t3
    global u1
    global u2
    global u3
    if marker3[offset3] == 0:
        return currentperm
    permutation = np.concatenate(([row[offset3] for row in rect3],currentperm))
    marker3[offset3] = 0
    offset2 = ((offset3 - t3 + u3)%u3 + t2)%u2
    return s2cylinder2(offset2, permutation)

def enum_strata2(n):
    '''
        Argument: integer n, size of origamis
        Return: array of origamis of the form type 1 in H(4) that have size n
        Comment: https://jamboard.google.com/d/1OSRaahlM2KiY1_onnQ5y6TDBOrseFGrHhtssKQc2ee0/edit?usp=sharing
            on slide 6 explains external gluing formula
    '''
    surfaces = []
    global u2
    for u2 in range(3, n - 1): # middle rectangle has width [2, n-1)
        for h2 in range(1, n // u2 + 1): # middle rectangle has height [1, n/u2)
            # top rectangle has width at most u2 - 2 or at most area left - 1
            global u1
            for u1 in range(1, min(u2 - 1, n - u2 * h2)):
                for h1 in range(1, (n - u2 * h2) // u1 + 1): # top rectangle has width [1, area)
                    global u3
                    for u3 in range(1, min(n - u2 * h2 - u1 * h1 + 1, u2 - u1)):
                        if (n - u2 * h2 - u1 * h1) % u3 == 0: # as long as u1 divides the area, it's valid
                            h3 = (n - u2 * h2 - u1 * h1) // u3
                            rectangle2 = np.arange(1, u2 * h2 + 1)
                            global rect2
                            rect2 = np.reshape(rectangle2, (h2, u2)) # base rectangle rect2
                            rect2 = np.flip(rect2,0)
                            rectangle1 = np.arange(u2 * h2 + 1, u2 * h2 + u1 * h1 + 1)
                            global rect1
                            rect1 = np.reshape(rectangle1, (h1, u1)) # top rectangle rect1
                            rect1 = np.flip(rect1,0)
                            rectangle3 = np.arange(u2 * h2 + u1 * h1 + 1, n + 1)
                            global rect3
                            rect3 = np.reshape(rectangle3, (h3, u3)) # top rectangle rect1
                            rect3 = np.flip(rect3,0)
                            hperm = '' 
                            # horizontal (right) permutation is independent of twist
                            for i in range(h1):
                                hperm += cycle_notation(rect1[i])
                            for i in range(h2):
                                hperm += cycle_notation(rect2[i])
                            for i in range(h3):
                                hperm += cycle_notation(rect3[i])
                            global t1
                            for t1 in range(u1): # singularity on cylinder 1
                                global t2
                                for t2 in range(u2 - u1): # singularity on cylinder 2
                                    global t3
                                    for t3 in range(u3):
                                        global marker1
                                        marker1 = [1] * u1 # marks columns in rect1 that have been visited by 0
                                        global marker2
                                        marker2 = [1] * u2
                                        global marker3
                                        marker3 = [1] * u3
                                        vperm = ''
                                        while sum(marker2) != 0:
                                            offset2 = np.nonzero(marker2)[0][0]
                                            column = []
                                            permutation = s2cylinder2(offset2, column)
                                            vperm += cycle_notation(np.flip(permutation,0))
                                        while sum(marker1) != 0:
                                            offset1 = np.nonzero(marker1)[0][0]
                                            column = []
                                            permutation = s2cylinder1(offset1, column)
                                            vperm += cycle_notation(np.flip(permutation,0))
                                        while sum(marker3) != 0:
                                            offset3 = np.nonzero(marker3)[0][0]
                                            column = []
                                            permutation = s2cylinder3(offset3, column)
                                            vperm += cycle_notation(np.flip(permutation,0))
                                        origami = Origami(hperm, vperm)
#                                         print(origami)
#                                         print(t1, t2, t3)
#                                         print(origami.stratum())
#                                         origami.show()
                                        surfaces.append(origami)
    return surfaces

In [9]:
# for i in range(17):
#     for ori in enum_strata2(i):
#         print(ori.stratum())